In [1]:
from directory_cobveg_treat import definir_diretorio
project_path = definir_diretorio()

import os
import geopandas as gpd

from utils import (
    get_data_diretorio,
    save_parquet_excel,
)

# Overlay Intersection entre área de Cobertura Vegetal e Distritos

In [2]:
data_path = get_data_diretorio(project_path)

# Dependências

Este notebook é dependente dos assets resultantes dos notebooks ../../arborizacao_viaria/carregar_dados/malha_distritos e load_data/malha_cobertura_vegetal

In [3]:
cobveg_path = os.path.join(
    data_path, 
    'assets', 
    'malha_cobertura_vegetal.parquet'
)
gdf_cobveg = gpd.read_parquet(cobveg_path)

In [4]:
distritos_path = os.path.join(data_path, 'assets', 'distrito_ibge.parquet')
gdf_distritos = gpd.read_parquet(distritos_path)

# Orvelay cobertura vegetal por Distrito

In [5]:
gdf_cobveg.sample(2)

,cd_cob_veg,cd_identi0,cd_categor,cd_subcate,cd_subcat0,qt_altura_,geometry
58534,58535.0,58708.0,13.0,0.0,0.0,0.0,"POLYGON ((326007.352 7398349.262, 326007.91 73..."
184606,184607.0,185084.0,14.0,0.0,0.0,0.0,"POLYGON ((330065.919 7380123.25, 330064.044 73..."


Como podemos ver na coluna de geometria, as áreas de cobertura vegetal são polígonos. Para calcular corretamente a quantidade de área verde por distritos, precisamos usar uma função chamada 'overlay intersection'.

Iniciamos registrado valor de cada polígono ANTES do processo de recorte com o overlay.

In [6]:
gdf_cobveg['area_cob_veg']=gdf_cobveg['geometry'].area

Agora utilizaremos a função do overlay:

In [ ]:
overlay_gdf = gpd.overlay(
    gdf_cobveg,
    gdf_distritos,
    how = 'intersection',
    keep_geom_type=True
)

Com esta função, caso um polígono de cobertura vegetal esteja dividido entre 2 ou mais distritos, eles serão divididos em diferente polígonos, recortados segundo os limites dos distritos. Mas, como os index (cd_cob_veg) se manterão, ainda poderemos saber quais poligonos correspondem a um poligono de cobertura vegetal anterior ao overlay.

In [ ]:
overlay_gdf['area_recort'] = overlay_gdf['geometry'].area
overlay_gdf['perc_recort'] = overlay_gdf['area_recort']/overlay_gdf['area_cob_veg']

In [ ]:
overlay_gdf.sample(2)

In [ ]:
gdf_cobveg['cd_cob_veg'].unique().shape

In [ ]:
overlay_gdf['cd_cob_veg'].unique().shape

In [ ]:
poligonos_omitidos = gdf_cobveg.loc[~gdf_cobveg['cd_cob_veg'].isin(overlay_gdf['cd_cob_veg'])]

In [ ]:
poligonos_omitidos.shape

In [ ]:
m = gdf_distritos.explore(color='pink', fillOpacity=0.1)
poligonos_omitidos.explore(
    m=m,
    color='aquamarine'
)

As 465 áreas de cobertura vegetal que foram omitidas estão fora dos limites dos distritos e, portanto, do município. Por conta disos, optamos por não adicioná-los no cálculo, mantendo-os de fora do overlay.

# Salvar gdf

In [ ]:
save_parquet_excel(
    gdf_distritos, 
    fname= 'overlay_cobertura_vegetal_distritos',
    data_path = data_path,
    data_subpath = os.path.join(
        'assets',
        'cobertura_vegetal'
    )
)